In [ ]:
#Helper for assigment

<h1> PLS NO TOUCH </h1>
<h3> Just execute </h3>

In [ ]:
#Imports
import sqlite3
high_performance_mode = False #Changed this to default false
db_file_path = "db.db" #CHANGE THIS
from urllib.parse import unquote
from urllib.parse import quote

<h4>For people that wanna do semantic analysis, everything u lookup you have to first encode with
quote("STRINGEXAMPLE") and then unquote("STRINGEXAMPLE") to read it</h4>
<h5>It is commonly used for encoding urls because its parser-friendly (kinda the reason why i used it to encode text that has very weird characters and to preserve its structure without encoding it using something like base64 which defeats the purpose of not loading into memory because then you can't do quick lookups over it)</h5>
<h5>There is a fast and a slow mode, i changed the default behaviour to slow, change it to fast by changing the boolean above this cell box, although its not recommended if you don't have atleast 8gb ram because it will dump to pagefile and slow down your OS and the program itself</h5>
And at the end, the helper functions are here for THE MOST BASIC functionality, everything else should be written in SQL, simplifying SQLite is not something thats a good use of time and you can find online resources like https://docs.mongodb.com/manual/reference/sql-comparison/ and automatic converters that allow you to use mongo queries and convert them to something you can execute via sql<br>
The reason we are not using something that has easily accessible data in every row is because of the size - deduplication is important and for us to be able to leverage the whole dataset (deduplication is the wrong word to use tho - because i didn't wanna make multiple dataset versions (because the duplicates existing is data that can be used for other analysis), there is still duplicate data, but its non-redundant (for more info peek @ https://en.wikipedia.org/wiki/Data_redundancy), we really need a faster way to look through it, that means that you sacrifice ease of use, and that means that you have to write intermediate-level sqlite queries to find the data, you ask one table a question and then use that data to evaluate another data, i specifically avoided using advanced sqlite just so that you can understand the queries you want (at the end, you can see that python is creating the queries themselves and executing them, thats why they are not sorted, but you can do it in pure sql and if somebody wants to do it, look @ https://www.sqlitetutorial.net/sqlite-subquery/

In [ ]:
##DO NOT TOUCH THESE
db = sqlite3.connect(":memory:")
db2 = sqlite3.connect(db_file_path)
##DO NOT TOUCH THESE
if high_performance_mode:
    db2.backup(db)
    db2.close()
else:
    db = db2
db.execute('PRAGMA journal_mode = MEMORY')
#Rollback journal in mem so that indexing is faster and crashes are catastrophic >:D
db.execute('PRAGMA cache_size = 536870912') #<- 512MB in B
#This is dinamically resized but the default is really low so let it grow!
db.execute('PRAGMA temp_store = MEMORY')
#Anything that needs to be generated, do it in mem because faster
db.execute('PRAGMA locking_mode = EXCLUSIVE')
#Remove lock/unlock overhead let it zoom
print("Loaded")



In [ ]:
#If a query is slow do this by replacing the id and tweets
#If a format string makes more sense to you it should look like
#"CREATE INDEX IF NOT EXISTS {0} ON {1} (0);".format(column_name, table_name)
#Indexes speed up db lookups, more detailed indexes can speed up how fast the db can return a result to you
###If you wanna learn more peek @ https://www.sqlitetutorial.net/sqlite-index/
db.execute('CREATE INDEX IF NOT EXISTS id ON tweets (id);')
#Creating indexes over everything is bad because, and i quote "Too many indexes create additional overhead associated with the extra amount of data pages that the Query Optimizer needs to go through."
#Those 

1. get_tweet(id, [vars])
2. get_tweet(id, [vars])
3. get_depth(id)
4. get_user_tweets(id, [vars])
5. get_vars(tweet_obj, [vars])
6. get_airline_tweets(airline_str, [vars])

In [ ]:
#Even tho the Helper functions exist - you'll just learn the lession about the tradeoff between
#Expresivity and simplicity
#Pls try and write the SQLite queries yourself
#Nice docs: https://www.sqlite.org/doclist.html
#Or just Google
c = db.cursor()
class Helper:
      def __init__(self, db):
        self.db = db;
        #If anybody wants to use this Google "SQLite IN"
        self.airline_names = {20626359: 'virginatlantic', 18332190: 'British Airways', 22536055: 'American Airlines', 1542862735: 'Ryanair', 38676903: 'easyJet', 124476322: 'Lufthansa', 218730857: 'Qantas', 106062176: 'Air France', 56377143: 'Royal Dutch Airlines', 45621423: 'SocialLogin%40etihad.ae', 253340062: 'Singapore Airlines', 26223583: 'airberlin', 2182373406: 'airberlin Assist'}
        self.airline_ids = [56377143, 106062176, 18332190, 22536055, 124476322, 26223583, 2182373406, 38676903, 1542862735, 253340062, 218730857, 45621423, 20626359]
      def exec_custom(self, query: str):
        return self.db.execute(query)
      def get_vars(self, table: str):
        return self.db.execute("pragma table_info('{0}');".format(table))
      def get_tables(self): 
        return self.db.execute("SELECT name FROM sqlite_master WHERE type ='table' AND name NOT LIKE 'sqlite_%';")
      def get_tweet(self, id: int, vars, filter):
        return self.db.execute(self.build_query('tweets', vars, filter))
      def get_user_tweets(self, id: int):
        return self.db.execute(self.build_query('tweets', '*', [['user', id]]))
      def get_airline_tweets_by_mention(self, id: int, vars):
        return self.db.execute(self.build_query('mentions', vars, "WHERE user_id = '{0}'".format(id)))
      def get_airline_tweets_by_user_id_in_tweet(self, id: int, vars):
        return self.db.execute(self.build_query('tweets', vars, "WHERE user = '{0}'".format(id)))
      def build_query(self, table, return_values, search_values, raw_filter = 0):
        def createLimit(limit: list):
            return limit[0] + " = " + "'{0}'".format(limit[1])
        #You'd have to sort and/or/* search values and then apply them accordingly because of how the .join function works and im not about that life
        #Or use the list comprehension output to process the logic behind it (but usually complex things require brackets and f that)
        if(type(search_values) == list):
            query = "select " + ",".join(return_values) + " from " + table + " where " + ' and '.join([createLimit(limit) for limit in search_values])
        else:
            query = "select " + ",".join(return_values) + " from " + table + " where " + search_values
        return query


In [ ]:
hp = Helper(c)

In [ ]:
#See all tables
hp_t = Helper(db)
for table_name in hp_t.get_tables():
    print('<[{0}]>'.format(table_name[0]))
    for column_name in hp_t.get_vars(table_name[0]):
        print("    {0}".format(column_name[1]))

In [ ]:
#ACTIVITY MENTION PER AIRLINE (Every place where the airline is mentioned)
#If u want to speed this up, add index to name and user_id in ent_mentions
query = '''
SELECT name, count(*) FROM ent_mentions WHERE user_id IN ({0}) GROUP BY user_id;
'''.format(
    (','.join([str(x) for x in hp.airline_ids])),
)
for row in hp.exec_custom(query):
    print(row)

In [ ]:
#NUMBER OF VALID CONVERSATIONS PER AIRLINE
c.execute('CREATE INDEX IF NOT EXISTS root ON convs (root);')
query = '''
SELECT count(DISTINCT root)
FROM convs
WHERE root
IN
(SELECT id FROM ent_mentions WHERE user_id = '{0}')
'''
ssum = 0
for id in hp.airline_ids:
    for row in hp.exec_custom(query.format(id)):
        print(hp.airline_names[id], " - ", row[0])
        ssum += row[0]
print("Sum: ", ssum)

In [ ]:
#MAX AND AVERAGE LENGTH PER AIRLINE
query = '''
SELECT max(level) as max_level, avg(level) AS average_level
FROM convs
WHERE root IN (SELECT id FROM ent_mentions WHERE user_id = '{0}')
'''
for id in hp.airline_ids:
    for row in hp.exec_custom(query.format(id)):
        print(hp.airline_names[id], row)

In [ ]:
#LANGUAGE PER AIRLINE
query = '''
SELECT language, count(language) as lang_c
from tweets
WHERE id IN (SELECT id FROM ent_mentions WHERE user_id = '{0}')
AND text != 'null'
GROUP BY language
ORDER BY lang_c DESC; 
'''
##
##Mentioned this as a possible problem, the "language detection" is as good as the grammar/spelling
##that the user has - could improve with additional removing of hashtags and other elements (altho
##looking through the code of the language "checker", it really should not matter)
##
for id in hp.airline_ids:
    print(hp.airline_names[id])
    for row in hp.exec_custom(query.format(id)):
        print(" ", row[0], "-", row[1])